# FeedBack Prize: EDA

## Requirments:

In [2]:
!pip install pandas

## Libraries

In [19]:
import spacy
import pandas as pd
from collections import Counter

## Configs

In [21]:
class CFG:
    # File paths:
    common_path = '/srv/dataset/datafbp'
    train_path = f'{common_path}/train.csv'
    test_path = f'{common_path}/test.csv'
    sub_path = f'{common_path}/sample_submission.csv'
    
    # Tokenizer:
    tokenizer = spacy.load('ru_core_news_sm', disable=['tok2vec', 'parser', 'attribute_ruler', 'ner'])

## Load dataframe

In [5]:
train_df = pd.read_csv(CFG.train_path)
test_df = pd.read_csv(CFG.test_path)

In [6]:
train_df

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5
...,...,...,...,...,...,...,...,...
3906,FFD29828A873,I believe using cellphones in class for educat...,2.5,3.0,3.0,3.5,2.5,2.5
3907,FFD9A83B0849,"Working alone, students do not have to argue w...",4.0,4.0,4.0,4.0,3.5,3.0
3908,FFDC4011AC9C,"""A problem is a chance for you to do your best...",2.5,3.0,3.0,3.0,3.5,3.0
3909,FFE16D704B16,Many people disagree with Albert Schweitzer's ...,4.0,4.5,4.5,4.0,4.5,4.5


## Text preprocessing (Lite)

In [ ]:
# Check list:
# \n\n
# Generic_City
# Jefferson

In [32]:
word_counter = Counter()
for text in train_df.full_text:
    text = CFG.tokenizer(text)
    tokens = [token for token in text]
    for word in tokens:
        word_counter.update([str(word)])

In [33]:
word_counter.most_common(10)

[('to', 71079),
 ('.', 69287),
 (',', 54804),
 ('the', 53063),
 ('and', 47105),
 ('you', 37758),
 ('a', 36007),
 ('that', 30050),
 ('is', 26333),
 ('in', 25831)]

In [44]:
n = 7
word_counter.most_common()[int(100 * n):int(100 * (n + 1))]

[('case', 240),
 ('likely', 239),
 ('either', 238),
 ('move', 238),
 ('safe', 238),
 ('child', 237),
 ('few', 236),
 ('telling', 236),
 ('sick', 236),
 ("'", 234),
 ('Jefferson', 234),
 ('gain', 233),
 ('fight', 233),
 ('teaching', 233),
 ('elective', 232),
 ('Like', 231),
 ('sometime', 231),
 ('Thomas', 231),
 ('Everyone', 229),
 ('open', 229),
 ('role', 229),
 ('interest', 229),
 ('4', 229),
 ('greatest', 228),
 ('says', 228),
 ('develop', 228),
 ('educational', 228),
 ('form', 228),
 ('usually', 227),
 ('multiple', 227),
 ('somebody', 227),
 ('living', 227),
 ("isn't", 226),
 ('assignments', 226),
 ('famous', 226),
 ('extracurricular', 226),
 ('small', 225),
 ('morning', 224),
 ('goes', 224),
 ('vacation', 224),
 ('comfortable', 223),
 ('communicate', 223),
 ('yes', 223),
 ('board', 223),
 ('course', 222),
 ('quote', 222),
 ('asked', 222),
 ('result', 221),
 ('chose', 221),
 ('saw', 221),
 ('lots', 220),
 ('interested', 220),
 ('collage', 219),
 (')', 218),
 ('watch', 217),
 ('On', 